
Exercise 5: use the client sever example from previous 
week to have a TCP server as DHCP server and create a client that would get 
the IP address from the server following the process above. 

In [ ]:
import socket
import threading

# Server Configuration
server = {
    "ip_pool": ["192.168.1.100", "192.168.1.101", "192.168.1.102"],
    "leases": {}
}

def handle_client(client_socket, client_address):
    print(f"\n[SERVER] Handling client {client_address}")
    
    data = client_socket.recv(1024).decode()
    print(f"[SERVER] Received: {data}")
    if data != "DISCOVER":
        print("[SERVER] Invalid request.")
        client_socket.close()
        return
    
    if not server["ip_pool"]:
        print("[SERVER] No IPs available!")
        client_socket.sendall("OFFER: No IP available".encode())
        client_socket.close()
        return
    
    offered_ip = server["ip_pool"].pop(0)
    print(f"[SERVER] Offering IP: {offered_ip}")
    client_socket.sendall(f"OFFER: {offered_ip}".encode())
    
    data = client_socket.recv(1024).decode()
    print(f"[SERVER] Received: {data}")
    if data != f"REQUEST: {offered_ip}":
        print("[SERVER] Invalid request.")
        client_socket.close()
        return
    
    server["leases"][client_address] = offered_ip
    print(f"[SERVER] ACKing IP {offered_ip} to {client_address}")
    client_socket.sendall(f"ACK: {offered_ip}".encode())
    
    print(f"[SERVER] IP Leases: {server['leases']}")
    
    client_socket.close()

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 65432))
server_socket.listen(5)

print("\n[SERVER] Waiting for client connections...")

def start_server():
    while True:
        client_socket, client_address = server_socket.accept()
        client_thread = threading.Thread(target=handle_client, args=(client_socket, client_address))
        client_thread.start()

if __name__ == "__main__":
    start_server()



[SERVER] Waiting for client connections...

[SERVER] Handling client ('127.0.0.1', 53423)
[SERVER] Received: DISCOVER
[SERVER] Offering IP: 192.168.1.100
[SERVER] Received: REQUEST: 192.168.1.100
[SERVER] ACKing IP 192.168.1.100 to ('127.0.0.1', 53423)
[SERVER] IP Leases: {('127.0.0.1', 53423): '192.168.1.100'}
